In [1]:
#notebook to match different name spellings and generate features for players

#import necessary packages
import pandas as pd
from fuzzywuzzy import fuzz, process

C:\Users\giuli\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#import data
#season 2019_20
PL2019_ratings = pd.read_csv('Data/2019-20/Players/PL2019_ratings.csv')
PL2019_fl = pd.read_csv('Data/2019-20/Players/FL_19.csv')
PL19_lineups = pd.read_csv('Data/2019-20/Players/PL2019_lineups.csv')
PL2019_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2019_fl['player'] = PL2019_fl['name']
#season 2020_21
PL2020_ratings = pd.read_csv('Data/2020-21/Players/PL2020_ratings.csv')
PL2020_fl = pd.read_csv('Data/2020-21/Players/FL_20.csv')
PL20_lineups = pd.read_csv('Data/2020-21/Players/PL2020_lineups.csv')
PL2020_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2020_fl['player'] = PL2020_fl['name']
#season 2020_21
PL2021_ratings = pd.read_csv('Data/2021-22/Players/PL2021_ratings.csv')
PL2021_fl = pd.read_csv('Data/2021-22/Players/FL_21.csv')
PL21_lineups = pd.read_csv('Data/2021-22/Players/PL2021_lineups.csv')
PL2021_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2021_fl['player'] = PL2021_fl['name']
PL2021_fl


,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player
0,Eric Bailly,DEF,Man Utd,0.0,0,0,0,0,0.0,286,...,0.0,0,0,0,0,50,True,0,1,Eric Bailly
1,Keinan Davis,FWD,Aston Villa,0.4,0,0,0,0,0.0,49,...,0.0,0,0,0,0,45,False,0,1,Keinan Davis
2,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,...,0.0,0,0,0,0,45,True,0,1,Ayotomiwa Dele-Bashiru
3,James Ward-Prowse,MID,Southampton,2.3,0,0,20,0,30.5,341,...,0.0,2,0,0,0,65,False,0,1,James Ward-Prowse
4,Bruno Miguel Borges Fernandes,MID,Man Utd,4.4,0,3,61,0,35.9,277,...,59.0,20,0,0,0,120,True,0,1,Bruno Miguel Borges Fernandes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25442,Wilfred Ndidi,MID,Leicester,0.0,0,0,0,0,0.0,216,...,0.0,0,-202,22,224,48,True,0,38,Wilfred Ndidi
25443,Matt Ritchie,DEF,Newcastle,0.9,0,0,3,0,0.0,292,...,0.0,1,143,396,253,49,False,0,38,Matt Ritchie
25444,Nathan Redmond,MID,Southampton,3.4,0,0,5,0,0.0,336,...,0.0,2,455,683,228,59,False,0,38,Nathan Redmond
25445,Mathew Ryan,GK,Brighton,0.0,0,0,0,0,0.0,65,...,0.0,0,-2,0,2,45,True,0,38,Mathew Ryan


In [3]:
#rename team names
PL2020_fl.replace(['Leeds', 'Leicester', 'Man City', 'Man Utd', 'Newcastle', 'Norwich','Spurs'], ['Leeds United', 'Leicester City','Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Norwich City' ,'Tottenham'], inplace=True)
PL2021_fl.replace(['Leeds', 'Leicester', 'Man City', 'Man Utd', 'Newcastle', 'Norwich','Spurs'], ['Leeds United', 'Leicester City','Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Norwich City' ,'Tottenham'], inplace=True)


In [4]:
#get name and team from df player ratings
players_rating = PL2021_ratings[['player', 'Team']]
players_rating = players_rating.drop_duplicates()
players_rating

,player,Team
0,Rico Henry,Brentford
1,Sergi Canós,Brentford
2,Ethan Pinnock,Brentford
3,Pontus Jansson,Brentford
4,Kristoffer Ajer,Brentford
...,...,...
10136,Alphonse Areola,West Ham United
10294,Kristoffer Klaesson,Leeds United
10444,Danny Ward,Leicester City
10449,John Ruddy,Wolverhampton Wanderers


In [5]:
#get name and team from df fantasy league
players_fl = PL2021_fl[['player', 'name', 'team']]
players_fl = players_fl.drop_duplicates()
players_fl

,player,name,team
0,Eric Bailly,Eric Bailly,Manchester Utd
1,Keinan Davis,Keinan Davis,Aston Villa
2,Ayotomiwa Dele-Bashiru,Ayotomiwa Dele-Bashiru,Watford
3,James Ward-Prowse,James Ward-Prowse,Southampton
4,Bruno Miguel Borges Fernandes,Bruno Miguel Borges Fernandes,Manchester Utd
...,...,...,...
23338,Adrian Blake,Adrian Blake,Watford
23815,Jack Grieves,Jack Grieves,Watford
24676,Joseph McGlynn,Joseph McGlynn,Burnley
24678,Anthony Mancini,Anthony Mancini,Burnley


In [6]:
#get name and team from df lineups
player_lineup = PL21_lineups[['player', 'team']]
player_lineup= player_lineup.drop_duplicates()
player_lineup['name_lineup'] = player_lineup['player']
player_lineup

,player,team,name_lineup
0,David Raya,Brentford,David Raya
1,Rico Henry,Brentford,Rico Henry
2,Ethan Pinnock,Brentford,Ethan Pinnock
3,Christian Nørgaard,Brentford,Christian Nørgaard
4,Sergi Canós,Brentford,Sergi Canós
...,...,...,...
14184,Jack Grieves,Watford,Jack Grieves
14320,Anthony Gomez Mancini,Burnley,Anthony Gomez Mancini
14324,Joseph McGlynn,Burnley,Joseph McGlynn
14642,Rhys Williams,Liverpool,Rhys Williams


In [8]:
#merge data
players_total = pd.merge(players_fl,player_lineup,on='player', how='left')
players_total = players_total.sort_values(by='name_lineup')

In [9]:
#remove blanks
player_lineup['player'] = player_lineup['player'].str.strip()
players_fl['player'] = players_fl['player'].str.strip()

#perform fuzzy merge
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

players_merged = fuzzy_merge(player_lineup, players_fl ,'player', 'player', threshold=90, limit=1)
players_merged

,player,team,name_lineup,matches
0,David Raya,Brentford,David Raya,David Raya Martin
1,Rico Henry,Brentford,Rico Henry,Rico Henry
2,Ethan Pinnock,Brentford,Ethan Pinnock,Ethan Pinnock
3,Christian Nørgaard,Brentford,Christian Nørgaard,Christian Nørgaard
4,Sergi Canós,Brentford,Sergi Canós,Sergi Canós
...,...,...,...,...
14184,Jack Grieves,Watford,Jack Grieves,Jack Grieves
14320,Anthony Gomez Mancini,Burnley,Anthony Gomez Mancini,Anthony Mancini
14324,Joseph McGlynn,Burnley,Joseph McGlynn,Joseph McGlynn
14642,Rhys Williams,Liverpool,Rhys Williams,Rhys Williams


In [10]:
#sort values to find names which could not be matched
players_merged= players_merged.sort_values(by='matches')
players_merged.reset_index(inplace=True, drop=True)
players_merged.rename(columns={"matches": "name"}, inplace=True)
players_merged

,player,team,name_lineup,name
0,Valentino Livramento,Southampton,Valentino Livramento,
1,Bernardo Silva,Manchester City,Bernardo Silva,
2,Cucho,Watford,Cucho,
3,Joe Willock,Newcastle United,Joe Willock,
4,Solly March,Brighton & Hove Albion,Solly March,
...,...,...,...,...
686,Zack Steffen,Manchester City,Zack Steffen,Zack Steffen
687,Zak Swanson,Arsenal,Zak Swanson,Zak Swanson
688,Álvaro Fernández,Brentford,Álvaro Fernández,Álvaro Fernández
689,Álvaro Fernández,Manchester United,Álvaro Fernández,Álvaro Fernández


In [11]:
#merge df with matches with fl df to find all unmatched players and manually match them in excel sheet

players_merged['player'] = players_merged['player'].str.strip()
players_fl['player'] = players_fl['player'].str.strip()

out = pd.merge(players_merged,players_fl , on='name', how='left')

out = out[['player_x', 'team_x', 'name', 'team_y']]
out.rename(columns={"player_x": "player_rating", "team_x": "team_rating", "name": "player_fl", "team_y": "team_fl" }, inplace=True)
out

,player_rating,team_rating,player_fl,team_fl
0,Valentino Livramento,Southampton,,NaN
1,Bernardo Silva,Manchester City,,NaN
2,Cucho,Watford,,NaN
3,Joe Willock,Newcastle United,,NaN
4,Solly March,Brighton & Hove Albion,,NaN
...,...,...,...,...
713,Álvaro Fernández,Brentford,Álvaro Fernández,Brentford
714,Álvaro Fernández,Brentford,Álvaro Fernández,Manchester Utd
715,Álvaro Fernández,Manchester United,Álvaro Fernández,Brentford
716,Álvaro Fernández,Manchester United,Álvaro Fernández,Manchester Utd


In [12]:
#export to excel
#out.to_excel('names_2021.xlsx')

In [19]:
#import manually added names
name_matching = pd.read_excel('Data/2021-22/Players/name_matching_2021.xlsx')
name_matching

,Unnamed: 0,player_rating,team_rating,player_fl,team_fl,names 20,names19
0,0,Valentino Livramento,Southampton,Tino Livramento,NaN,Valentino Livramento,NaN
1,1,Bernardo Silva,Manchester City,Bernardo Mota Veiga de Carvalho e Silva,NaN,Bernardo Mota Veiga de Carvalho e Silva,Bernardo Mota Veiga de Carvalho e Silva
2,2,Cucho,Watford,Juan Camilo Hernández Suárez,NaN,NaN,NaN
3,3,Joe Willock,Newcastle United,Joseph Willock,NaN,Joseph Willock,Joseph Willock
4,4,Solly March,Brighton & Hove Albion,Solomon March,NaN,Solomon March,Solomon March
...,...,...,...,...,...,...,...
713,713,Álvaro Fernández,Brentford,Álvaro Fernández,Brentford,NaN,NaN
714,714,Álvaro Fernández,Brentford,Álvaro Fernández,Manchester Utd,NaN,NaN
715,715,Álvaro Fernández,Manchester United,Álvaro Fernández,Brentford,NaN,NaN
716,716,Álvaro Fernández,Manchester United,Álvaro Fernández,Manchester Utd,NaN,NaN


In [20]:
#transform data for merge
name_matching = name_matching[['player_rating', 'player_fl']]
name_matching = name_matching.drop_duplicates()
name_matching.rename(columns={"player_fl": "player"}, inplace=True)

In [21]:
#get date out of kickoff time
PL2021_fl ['date'] = PL2021_fl['kickoff_time'].str.slice(start=0, stop=10)

In [22]:
#merge data
PL2021_fl['player'] = PL2021_fl['player'].str.strip()
name_matching['player'] = name_matching['player'].str.strip()
PL2021_fl_matched = pd.merge(PL2021_fl, name_matching , on='player', how='left')
#generate ID to map each time a player played a game to a specific
PL2021_fl_matched['ID'] = PL2021_fl_matched['player_rating'] + '_' + PL2021_fl_matched['date'].astype(str)

In [23]:
#generate ID
PL2021_ratings['ID'] = PL2021_ratings['player'] + '_' + PL2021_ratings['date'].astype(str)

In [24]:
#merge data on id, to get player rating for every game
PL2021_matched = pd.merge(PL2021_fl_matched, PL2021_ratings , on='ID', how='left')

In [25]:
#replace na (If a player was not in the squad for the match day, he will not receive a rating)
PL2021_matched['Rating'] = PL2021_matched['Rating'].fillna(0)
PL2021_matched

,name,position_x,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,ID,player_y,position_y,age,minutes_y,Team,matchweek_y,category,Rating,date_y
0,Eric Bailly,DEF,Manchester Utd,0.0,0,0,0,0,0.0,286,...,Eric Bailly_2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
1,Keinan Davis,FWD,Aston Villa,0.4,0,0,0,0,0.0,49,...,Keinan Davis_2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
2,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3,James Ward-Prowse,MID,Southampton,2.3,0,0,20,0,30.5,341,...,James Ward-Prowse_2021-08-14,James Ward-Prowse,CM,26-286,90.0,Southampton,1.0,Midfield,28.509599,2021-08-14
4,Bruno Miguel Borges Fernandes,MID,Manchester Utd,4.4,0,3,61,0,35.9,277,...,Bruno Fernandes_2021-08-14,Bruno Fernandes,AM,26-340,90.0,Manchester United,1.0,Midfield,51.721369,2021-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25442,Wilfred Ndidi,MID,Leicester City,0.0,0,0,0,0,0.0,216,...,Wilfred Ndidi_2022-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
25443,Matt Ritchie,DEF,Newcastle Utd,0.9,0,0,3,0,0.0,292,...,Matt Ritchie_2022-05-22,Matt Ritchie,LW,32-254,2.0,Newcastle United,38.0,Forward,4.998869,2022-05-22
25444,Nathan Redmond,MID,Southampton,3.4,0,0,5,0,0.0,336,...,Nathan Redmond_2022-05-22,Nathan Redmond,FW,28-077,82.0,Southampton,38.0,Forward,19.069354,2022-05-22
25445,Mathew Ryan,GK,Brighton,0.0,0,0,0,0,0.0,65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


In [26]:
#rename column names
PL2021_total = PL2021_matched[['player_x','player_rating','date_x', 'team' ,'Team' ,'category', 'minutes_x' ,'minutes_y','Rating', 'bonus', 'bps', 'xP' ,'ict_index', 'influence', 'creativity','threat','total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home', 'opponent_team', 'selected' ]]
PL2021_total.rename(columns={"player_x": "player_fl","player_rating": "player" ,"date_x": "date", "Team": "team_rating","team":"team",  "position_x": "position_fl", "minutes_x": "minutes_fl","minutes_y": "minutes_rating" ,"Rating": "rating"}, inplace=True)
PL2021_total


C:\Users\giuli\AppData\Local\Temp\ipykernel_113232\1640504712.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2021_total.rename(columns={"player_x": "player_fl","player_rating": "player" ,"date_x": "date", "Team": "team_rating","team":"team",  "position_x": "position_fl", "minutes_x": "minutes_fl","minutes_y": "minutes_rating" ,"Rating": "rating"}, inplace=True)


,player_fl,player,date,team,team_rating,category,minutes_fl,minutes_rating,rating,bonus,...,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected
0,Eric Bailly,Eric Bailly,2021-08-14,Manchester Utd,NaN,NaN,0,NaN,0.000000,0,...,0.0,0.0,0,0,0,0,50,True,10,9363
1,Keinan Davis,Keinan Davis,2021-08-14,Aston Villa,NaN,NaN,0,NaN,0.000000,0,...,0.0,0.0,0,0,0,0,45,False,18,169789
2,Ayotomiwa Dele-Bashiru,NaN,2021-08-14,Watford,NaN,NaN,0,NaN,0.000000,0,...,0.0,0.0,0,0,0,0,45,True,2,4092
3,James Ward-Prowse,James Ward-Prowse,2021-08-14,Southampton,Southampton,Midfield,90,90.0,28.509599,0,...,30.5,0.0,2,0,0,0,65,False,8,299682
4,Bruno Miguel Borges Fernandes,Bruno Fernandes,2021-08-14,Manchester Utd,Manchester United,Midfield,90,90.0,51.721369,3,...,35.9,59.0,20,0,0,0,120,True,10,3381004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25442,Wilfred Ndidi,Wilfred Ndidi,2022-05-22,Leicester City,NaN,NaN,0,NaN,0.000000,0,...,0.0,0.0,0,-202,22,224,48,True,16,70366
25443,Matt Ritchie,Matt Ritchie,2022-05-22,Newcastle Utd,Newcastle United,Forward,1,2.0,4.998869,0,...,0.0,0.0,1,143,396,253,49,False,5,15999
25444,Nathan Redmond,Nathan Redmond,2022-05-22,Southampton,Southampton,Forward,82,82.0,19.069354,0,...,0.0,0.0,2,455,683,228,59,False,9,13856
25445,Mathew Ryan,NaN,2022-05-22,Brighton,NaN,NaN,0,NaN,0.000000,0,...,0.0,0.0,0,-2,0,2,45,True,19,587


In [27]:
#generate game and player ID
PL2021_total['player_ID'] = PL2021_total['player'].astype(str) + PL2021_total['team_rating'] + PL2021_total['date'].astype(str)
PL2021_total['game_ID'] = PL2021_total['team_rating'] + PL2021_total['date'].astype(str)

C:\Users\giuli\AppData\Local\Temp\ipykernel_113232\947323803.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2021_total['player_ID'] = PL2021_total['player'].astype(str) + PL2021_total['team_rating'] + PL2021_total['date'].astype(str)
C:\Users\giuli\AppData\Local\Temp\ipykernel_113232\947323803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2021_total['game_ID'] = PL2021_total['team_rating'] + PL2021_total['date'].astype(str)


In [28]:
PL2021_total

,player_fl,player,date,team,team_rating,category,minutes_fl,minutes_rating,rating,bonus,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected,player_ID,game_ID
0,Eric Bailly,Eric Bailly,2021-08-14,Manchester Utd,NaN,NaN,0,NaN,0.000000,0,...,0,0,0,0,50,True,10,9363,NaN,NaN
1,Keinan Davis,Keinan Davis,2021-08-14,Aston Villa,NaN,NaN,0,NaN,0.000000,0,...,0,0,0,0,45,False,18,169789,NaN,NaN
2,Ayotomiwa Dele-Bashiru,NaN,2021-08-14,Watford,NaN,NaN,0,NaN,0.000000,0,...,0,0,0,0,45,True,2,4092,NaN,NaN
3,James Ward-Prowse,James Ward-Prowse,2021-08-14,Southampton,Southampton,Midfield,90,90.0,28.509599,0,...,2,0,0,0,65,False,8,299682,James Ward-ProwseSouthampton2021-08-14,Southampton2021-08-14
4,Bruno Miguel Borges Fernandes,Bruno Fernandes,2021-08-14,Manchester Utd,Manchester United,Midfield,90,90.0,51.721369,3,...,20,0,0,0,120,True,10,3381004,Bruno FernandesManchester United2021-08-14,Manchester United2021-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25442,Wilfred Ndidi,Wilfred Ndidi,2022-05-22,Leicester City,NaN,NaN,0,NaN,0.000000,0,...,0,-202,22,224,48,True,16,70366,NaN,NaN
25443,Matt Ritchie,Matt Ritchie,2022-05-22,Newcastle Utd,Newcastle United,Forward,1,2.0,4.998869,0,...,1,143,396,253,49,False,5,15999,Matt RitchieNewcastle United2022-05-22,Newcastle United2022-05-22
25444,Nathan Redmond,Nathan Redmond,2022-05-22,Southampton,Southampton,Forward,82,82.0,19.069354,0,...,2,455,683,228,59,False,9,13856,Nathan RedmondSouthampton2022-05-22,Southampton2022-05-22
25445,Mathew Ryan,NaN,2022-05-22,Brighton,NaN,NaN,0,NaN,0.000000,0,...,0,-2,0,2,45,True,19,587,NaN,NaN


In [29]:
#export player dataframe including fantasy league values and calculated ratings
#PL2021_total.to_csv('Data/2021-22/Players/PL2021_total.csv')